# Concomitant Medication vs Medical History Reconciliation using GPT

In [1]:
# Import the necessary libraries, modules and functions:
import pandas as pd
import numpy as np
import openai
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
from datetime import datetime
from getpass import getpass

# Prompt to enter OpenAI API key:
openai.api_key = getpass()

········


In [2]:
# Load the CM dataset as a dataframe:
dtypes1 = {'CENTRE': str, 'SUBJECT_ID': str, 'FORM_OCCURENCE': str, 'CMTRT': str, 'CMINDC': str, 'CMINDC_CAT': str, 'CMONGO': str, 'CMSTDAT_d': str, 'CMSTDAT_m': str, 'CMSTDAT_y': str, 'CMENDAT_d': str, 'CMENDAT_m': str, 'CMENDAT_y': str}
df1 = pd.read_csv('Input (EDC Datasets)\CM.csv', dtype = dtypes1)

# Fill all blank CM start and end date parts to their theoretical extremes. Missing data edit checks in the EDC will generally
# avoid the need for this, however blank dates may still occur on occasion. This allows column-wise date comparisons to still
# compute later on without error if blank dates are present:
df1['CMSTDAT_d'] = df1['CMSTDAT_d'].fillna('01')
df1['CMSTDAT_m'] = df1['CMSTDAT_m'].fillna('01')
df1['CMSTDAT_y'] = df1['CMSTDAT_y'].fillna('0001')
df1['CMENDAT_d'] = df1['CMENDAT_d'].fillna('31')
df1['CMENDAT_m'] = df1['CMENDAT_m'].fillna('12')
df1['CMENDAT_y'] = df1['CMENDAT_y'].fillna('9999')

# Define and apply a function to pad CM day and month parts with a leading zero if not already present. This ensures the
# concatenated date format will be accepted by the later date comparison function. This may or may not be necessary for your
# specific scenario:
def pad_with_zero(x):
    if pd.isnull(x):
        return x
    elif len(x) == 1:
        return x.zfill(2)
    else:
        return x
df1['CMSTDAT_d'] = df1['CMSTDAT_d'].apply(pad_with_zero)
df1['CMSTDAT_m'] = df1['CMSTDAT_m'].apply(pad_with_zero)
df1['CMENDAT_d'] = df1['CMENDAT_d'].apply(pad_with_zero)
df1['CMENDAT_m'] = df1['CMENDAT_m'].apply(pad_with_zero)

# If your CM dataset allows partially unknown dates, replace the raw EDC codelist values representing the unknown date parts
# with standard 'UNK' notation. This ensures the unknown date format will be accepted by the later date comparison function.
# This assumes your EDC platform captures the day, month and year parts as separate fields, otherwise equivalent operations can
# be performed on already combined date strings using regex:
df1['CMSTDAT_d'] = df1['CMSTDAT_d'].replace('UN', 'UNK')
df1['CMSTDAT_m'] = df1['CMSTDAT_m'].replace('UNK', 'UNK')
df1['CMSTDAT_y'] = df1['CMSTDAT_y'].replace('UKUK', 'UNK')
df1['CMENDAT_d'] = df1['CMENDAT_d'].replace('UN', 'UNK')
df1['CMENDAT_m'] = df1['CMENDAT_m'].replace('UNK', 'UNK')
df1['CMENDAT_y'] = df1['CMENDAT_y'].replace('UKUK', 'UNK')

# Concatenate the CM day, month and year parts into a single date string. This may or may not be necessary, depending upon your
# EDC's method of capturing partially unknown dates:
df1 = df1.drop(['CMSTDAT', 'CMENDAT'], axis = 1)
df1['CMSTDAT'] = df1['CMSTDAT_d'] + '-' + df1['CMSTDAT_m'] + '-' + df1['CMSTDAT_y']
df1['CMENDAT'] = df1['CMENDAT_d'] + '-' + df1['CMENDAT_m'] + '-' + df1['CMENDAT_y']

# Show only the necessary CM columns for the reconciliation and then filter the CM dataframe so that it only contains CMs where
# the indication category is 'MH'. Fill all empty CMINDC fields with a blank space so the 'get_embedding' function won't throw
# an error later on:
df1 = df1[['CENTRE', 'SUBJECT_ID', 'FORM_OCCURENCE', 'CMTRT', 'CMINDC', 'CMINDC_CAT', 'CMSTDAT', 'CMONGO', 'CMENDAT']]
df1 = df1.rename(columns = {'FORM_OCCURENCE': 'CM#'})
df1 = df1[df1['CMINDC_CAT'] == 'MH'].reset_index(drop = True)
df1['CMINDC'] = df1['CMINDC'].fillna(" ")

# Get the LLM vector embeddings for all CM indications. Vector embeddings are numerical representations of the underlying
# meaning (semantics) of natural language. This allows the program to perform computations on free text entries in the CM
# indication field. With the code below, only a list of isolated CM indications will be sent to OpenAI, with no other context:
df1['CMINDC_Embedding'] = df1['CMINDC'].apply(lambda x: get_embedding(x, engine = 'text-embedding-ada-002'))

# Load the MH dataset as a dataframe and drop rows where MHTERM is empty:
dtypes2 = {'CENTRE': str, 'SUBJECT_ID': str, 'FORM_OCCURENCE': str, 'MHTERM': str, 'MHCAT': str, 'MHONGO': str, 'MHSTDAT_d': str, 'MHSTDAT_m': str, 'MHSTDAT_y': str, 'MHENDAT_d': str, 'MHENDAT_m': str, 'MHENDAT_y': str}
df2 = pd.read_csv('Input (EDC Datasets)\MH.csv', dtype = dtypes2)
df2 = df2.dropna(subset = ['MHTERM'])

# For MH items that are procedures, set the blank end date values to be equal to the start date values. This is only applicable
# if your MH EDC form / dataset captures prior procedures together with MH (rather than in PR) and the end date fields therefore
# become dynamically hidden due to them being procedures:
df2.loc[df2['MHCAT'] == 'PR', 'MHENDAT_d'] = df2.loc[df2['MHCAT'] == 'PR', 'MHSTDAT_d']
df2.loc[df2['MHCAT'] == 'PR', 'MHENDAT_m'] = df2.loc[df2['MHCAT'] == 'PR', 'MHSTDAT_m']
df2.loc[df2['MHCAT'] == 'PR', 'MHENDAT_y'] = df2.loc[df2['MHCAT'] == 'PR', 'MHSTDAT_y']

# Fill all blank MH start and end date parts to their theoretical extremes. Missing data edit checks in the EDC will generally
# avoid the need for this, however blank dates may still occur on occasion. This allows column-wise date comparisons to still
# compute later on without error if blank dates are present:
df2['MHSTDAT_d'] = df2['MHSTDAT_d'].fillna('01')
df2['MHSTDAT_m'] = df2['MHSTDAT_m'].fillna('01')
df2['MHSTDAT_y'] = df2['MHSTDAT_y'].fillna('0001')
df2['MHENDAT_d'] = df2['MHENDAT_d'].fillna('31')
df2['MHENDAT_m'] = df2['MHENDAT_m'].fillna('12')
df2['MHENDAT_y'] = df2['MHENDAT_y'].fillna('9999')

# Apply the 'pad_with_zero' function to pad MH day and month parts with a leading zero if not already present. This ensures the
# concatenated date format will be accepted by the later date comparison function. This may or may not be necessary for your
# specific scenario:
df2['MHSTDAT_d'] = df2['MHSTDAT_d'].apply(pad_with_zero)
df2['MHSTDAT_m'] = df2['MHSTDAT_m'].apply(pad_with_zero)
df2['MHENDAT_d'] = df2['MHENDAT_d'].apply(pad_with_zero)
df2['MHENDAT_m'] = df2['MHENDAT_m'].apply(pad_with_zero)

# If your MH dataset allows partially unknown dates, replace the raw EDC codelist values representing the unknown date parts
# with standard 'UNK' notation. This ensures the unknown date format will be accepted by the later date comparison function.
# This assumes your EDC platform captures the day, month and year parts as separate fields, otherwise equivalent operations can
# be performed on already combined date strings using regex:
df2['MHSTDAT_d'] = df2['MHSTDAT_d'].replace('UN', 'UNK')
df2['MHSTDAT_m'] = df2['MHSTDAT_m'].replace('UNK', 'UNK')
df2['MHSTDAT_y'] = df2['MHSTDAT_y'].replace('UKUK', 'UNK')
df2['MHENDAT_d'] = df2['MHENDAT_d'].replace('UN', 'UNK')
df2['MHENDAT_m'] = df2['MHENDAT_m'].replace('UNK', 'UNK')
df2['MHENDAT_y'] = df2['MHENDAT_y'].replace('UKUK', 'UNK')

# Concatenate the MH day, month and year parts into a single date string. This may or may not be necessary, depending upon your
# EDC's method of capturing partially unknown dates:
df2 = df2.drop(['MHSTDAT', 'MHENDAT'], axis = 1)
df2['MHSTDAT'] = df2['MHSTDAT_d'] + '-' + df2['MHSTDAT_m'] + '-' + df2['MHSTDAT_y']
df2['MHENDAT'] = df2['MHENDAT_d'] + '-' + df2['MHENDAT_m'] + '-' + df2['MHENDAT_y']

# Show only the necessary MH columns for the reconciliation:
df2 = df2[['CENTRE', 'SUBJECT_ID', 'FORM_OCCURENCE', 'MHTERM', 'MHCAT', 'MHSTDAT', 'MHONGO', 'MHENDAT']]
df2 = df2.rename(columns = {'FORM_OCCURENCE': 'MH#'})

# Get the LLM vector embeddings for all MH terms. Vector embeddings are numerical representations of the underlying meaning
# (semantics) of natural language. This allows the program to perform computations on free text entries in the MH term field.
# With the code below, only a list of isolated MH terms will be sent to OpenAI, with no other context:
df2['MHTERM_Embedding'] = df2['MHTERM'].apply(lambda x: get_embedding(x, engine = 'text-embedding-ada-002'))

In [3]:
# Perform a left join on the CM (left) and MH (right) dataframes using Centre and Subject_ID as the keys. This combines the CM
# and MH data into a single dataframe, providing all pairwise combinations of CMs and MHs side-by-side within each subject:
df3 = pd.merge(df1, df2, on = ['CENTRE', 'SUBJECT_ID'], how = 'left')

# Calculate the cosine similarity between the CM indication and MH term embedding vectors. This represents the difference in
# angle between the CM indication and MH term embedding vectors in n-dimensional space and numerically represents the semantic
# similarity between the two pieces of text:
df3['Similarity'] = df3.apply(lambda row: cosine_similarity(row['CMINDC_Embedding'], row['MHTERM_Embedding']) if pd.notnull(row['MH#']) else 0, axis = 1)

# Identify all CM-MH pairs where the cosine similarity is sufficiently high to indicate a semantic match. Tweak the similarity
# threshold to meet your specific needs:
df3['Similarity_Match'] = df3['Similarity'] >= 0.95

In [4]:
# Define a function to check if a date, which may be partially unknown, falls within a specified known date range. This function
# is used to check if both the CM start and end dates fall within the timeframe of each MH. It requires the input date strings
# to be in a dd-mm-yyyy format (e.g. 01-01-2023). It first determines if the input date_str has any unknown parts, and then
# varies the unknown parts to encompass all possible values they could take, otherwise it keeps just the known part. The function
# then consecutively iterates over all combinations of day, month and year part ranges in a nested manner, combining them into
# single datetime objects which are then evaluated as either True or False according to: start_date <= date <= end_date. If at
# least one of the possible dates yields a True result (i.e. is within range), then the loop stops and the output of the
# function is True, otherwise the function continues until it reaches the end of its nested loops, where it will output False:

def is_within_range(start_date_str, end_date_str, date_str):
    start_date_formats = ['%d-%m-%Y', '%d-%b-%Y']
    end_date_formats = ['%d-%m-%Y', '%d-%b-%Y']

    for start_format in start_date_formats:
        try:
            start_date = datetime.strptime(start_date_str, start_format)
            break
        except ValueError:
            pass
    else:
        raise ValueError(f"Invalid start date format: {start_date_str}")

    for end_format in end_date_formats:
        try:
            end_date = datetime.strptime(end_date_str, end_format)
            break
        except ValueError:
            pass
    else:
        raise ValueError(f"Invalid end date format: {end_date_str}")

    date_parts = date_str.split('-')
    day_range = range(1, 32) if date_parts[0] == 'UNK' else [int(date_parts[0])]
    month_range = range(1, 13) if date_parts[1] == 'UNK' else [int(date_parts[1])]
    year_range = range(start_date.year, end_date.year + 1) if date_parts[2] == 'UNK' else [int(date_parts[2])]

    for day in day_range:
        for month in month_range:
            for year in year_range:
                try:
                    date = datetime(day=day, month=month, year=year)
                    if start_date <= date <= end_date:
                        return True
                except ValueError:
                    pass

    return False

In [5]:
# Fill any post-merge 'NaN' MH dates with a dummy date so that the 'is_within_range' function won't throw an error:
df3['MHSTDAT'] = df3['MHSTDAT'].fillna('01-01-0001')
df3['MHENDAT'] = df3['MHENDAT'].fillna('01-01-0001')

# Set all unknown MH date parts to their lowermost or uppermost possible extremes, depending upon whether they represent a start
# date or end date:
df3['MHSTDAT'] = df3['MHSTDAT'].str.replace('^UNK-', '01-', regex = True).str.replace('-UNK-', '-01-').str.replace('-UNK$', '-0001', regex = True)
df3['MHENDAT'] = df3['MHENDAT'].str.replace('^UNK-UNK-', '31-12-', regex = True)
df3['MHENDAT'] = df3['MHENDAT'].str.replace('-UNK-', '-12-').str.replace('-UNK$', '-9999', regex = True)
df3['MHENDAT'] = df3['MHENDAT'].str.replace(r'^(UNK-)(02-)', r'28-\2', regex = True).str.replace(r'^(UNK-)(04-|06-|09-|11-)', r'30-\2', regex = True).str.replace(r'^(UNK-)(01-|03-|05-|07-|08-|10-|12-)', r'31-\2', regex = True)

if not df3.empty:
    
    # Determine if the CM start and end dates are within (or possible to be within) the timeframe of each MH by applying the
    # 'is_within_range' function:
    df3['CMSTDAT_Within_Range'] = df3.apply(lambda row: is_within_range(row['MHSTDAT'], row['MHENDAT'], row['CMSTDAT']), axis = 1)
    df3['CMENDAT_Within_Range'] = df3.apply(lambda row: is_within_range(row['MHSTDAT'], row['MHENDAT'], row['CMENDAT']), axis = 1)

In [6]:
if not df3.empty:
    
    # Determine if the CM indication and MH term semantically match AND the CM start date is within the MH timeframe AND the CM
    # end date is within the MH timeframe. If all of these conditions are met, then the 'Entire_Match' column value is True,
    # otherwise it is False:
    df3['Entire_Match'] = df3['Similarity_Match'] & df3['CMSTDAT_Within_Range'] & df3['CMENDAT_Within_Range']
    
    # Determine if each Subject-CM grouping has a fully reconciling MH present. If a reconciling MH is present, then the
    # 'Entire_Match_Grouped' column value is True, otherwise it is False:
    df3['Entire_Match_Grouped'] = df3.groupby(['CENTRE', 'SUBJECT_ID', 'CM#', 'CMTRT', 'CMINDC'])['Entire_Match'].transform(lambda x: True if x.sum() else False)
    
    # Filter the dataframe so that it contains only Subject-CMs that don't have a fully reconciling MH present:
    df4 = df3[df3['Entire_Match_Grouped'] == False].reset_index(drop = True)
    
    # Determine if the Subject-CMs that don't have a fully reconciling MH present, have at least one semantically matching MH
    # term present (i.e. partial reconciliation of the MH term, but not dates). If a partially reconciling MH is present, then
    # the 'Similarity_Match_Grouped' column value is True, otherwise it is False:
    df4['Similarity_Match_Grouped'] = df4.groupby(['CENTRE', 'SUBJECT_ID', 'CM#', 'CMTRT', 'CMINDC'])['Similarity_Match'].transform(lambda x: True if x.sum() else False)
    
    if not df4.empty:
        
        # Create a new column called 'ISSUE_DESCRIPTION' containing a descriptive message for the check's output which depends
        # upon the value in the 'Similarity_Match_Grouped' column:
        df4.loc[df4['Similarity_Match_Grouped'] == True, 'ISSUE_DESCRIPTION'] = "Matching term found in MH, but dates inconsistent"
        df4.loc[df4['Similarity_Match_Grouped'] == False, 'ISSUE_DESCRIPTION'] = "No matching term found in MH"
        
        # Show only the unique Subject-CMs for which there are flagged reconciliation issues:
        df5 = df4.groupby(['CENTRE', 'SUBJECT_ID', 'CM#', 'CMTRT', 'CMINDC', 'CMINDC_CAT', 'ISSUE_DESCRIPTION'], as_index = False).agg({'MH#': 'count'}).drop('MH#', axis = 1)
        
    else:
        df5 = pd.DataFrame({"No Issues Found": []})
else:
    df5 = pd.DataFrame({"No Issues Found": []})
df5

,CENTRE,SUBJECT_ID,CM#,CMTRT,CMINDC,CMINDC_CAT,ISSUE_DESCRIPTION
0,999,S009,1,Fluconazole,Fungal skin infection,MH,No matching term found in MH
1,999,S010,1,Codeine,Hysterectomy pain,MH,No matching term found in MH
2,999,S011,1,Codeine,Mastectomy pain,MH,No matching term found in MH
3,999,S012,1,Codeine,Rhinoplasty pain,MH,No matching term found in MH
4,999,S013,1,Codeine,Rhytidectomy pain,MH,No matching term found in MH
5,999,S014,1,Paracetamol,Headache,MH,"Matching term found in MH, but dates inconsistent"
6,999,S016,1,Zolpidem,Insomnia,MH,No matching term found in MH
7,999,S017,1,Loratadine,Hayfever,MH,"Matching term found in MH, but dates inconsistent"
8,999,S019,1,Paracetamol,Covid-19,MH,"Matching term found in MH, but dates inconsistent"


In [7]:
# Lastly, export the output to an excel spreadsheet:
df5.to_excel('Output\CM vs MH Reconciliation Output.xlsx', sheet_name = 'CM vs MH Reconciliation', startrow = 0, startcol = 0, index = False, na_rep = '', header = True)